# Домашнее задание

### Д/з из четырех пунктов:
* Улучшение `fit_generator`
* Сравнение двух ReLU (разные активации)
* Испорченный батч-норм 
* "Сырые" данные. 

### Что нужно сделать
* Следовать инструкциям в каждом из пунктов.
* Результатами вашей работы будет ноутбук с доработанным кодом + архив с директорией с логами `tensorboard` `logs/`, в который вы запишите результаты экспериментов. Подробности в инструкциях ниже.
* Можно и нужно пользоваться кодом из файла `utils`, **но** весь код модифицируйте, пожалуйста, в ноутбуках! Так мне будет проще проверять.

**Загрузка tensorboard в ноутбук**

Можете попробовать использовать его так на свой страх и риск :)

In [1]:
# %load_ext tensorboard
# %tensorboard --logdir logs

**Импорты**

In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
from typing import Callable

### Импорт слоев для д/з

In [3]:
from utils2 import BatchNormFlawed, Dense, DenseSmart, Sequential, MNISTSequence

### Загрузка данных

> Здесь ничего менять не нужно

In [4]:
(X_tr, y_tr), (X_test, y_test) = keras.datasets.mnist.load_data()

In [5]:
train_seq = MNISTSequence(X_tr, y_tr, 128)
test_seq = MNISTSequence(X_test, y_test, 128)

**Очистка данных**

In [6]:
# !rm -rf logs/*

In [7]:
# !mkdir -p logs

In [8]:
keras.backend.clear_session()

## 1. Улучшение fit_generator

Улучшите метод `fit_generator` так, чтобы он:
* Записывал значения градиентов для всех переменных при помощи `tf.summary.histogram` 
* Записывал значения ошибки и метрики на валидации с помощью `tf.summary.scalar`

Затем сделайте monkey patch класса sequential обновленным методом (следующая ячейка за методом `fit_generator`).

In [9]:
def fit_generator(self, train_seq, eval_seq, epoch, loss, optimizer, writer=None):
    history = dict(train=list(), val=list())

    train_loss_results = list()
    val_loss_results = list()

    train_accuracy_results = list()
    val_accuracy_results = list()

    step = 0
    for e in range(epoch):
        p = tf.keras.metrics.Mean()
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_loss_avg_val = tf.keras.metrics.Mean()

        epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
        epoch_accuracy_val = tf.keras.metrics.SparseCategoricalAccuracy()

        for x, y in train_seq:
            with tf.GradientTape() as tape:
                """
                Обратите внимание! Если записывать гистограмму каждый шаг,
                обучение будет идти очень медленно. Поэтому записываем данные 
                каждый i-й шаг.
                """
                if step % 50 == 0:
                    prediction = self._forward(x, writer, step)
                else:
                    prediction = self._forward(x)
                loss_value = loss(y, prediction)
                    
            ###############################################################
            #                                                             #
            # Добавьте запись градиентов в гистограммы                    #
            #                                                             #
            ###############################################################
            gradients = tape.gradient(loss_value, self._trainable_variables)
               
            if step % 50 == 0:
                """
                Пример того, как можно дать всем градиентам уникальные имена. 
                Обратите внимание! Создание grad_names лучше вынести из цикла,
                чтобы не пересоздавать список на каждом шаге! 
                """
                grad_names = list()
                for layer in self._layers:          
                    for var_num, var in enumerate(layer.get_trainable()):
                        grad_names.append(f"grad_{layer.name}_{var_num}")
                zipped_gradients = zip(grad_names, gradients)
                with writer.as_default():
                    for k, v in zipped_gradients:
                        tf.summary.histogram(k, v, step=step)
                        
            optimizer.apply_gradients(zip(gradients, self._trainable_variables))
            epoch_accuracy.update_state(y, prediction)
            epoch_loss_avg.update_state(loss_value)

            if step % 50 == 0:
                with writer.as_default():
                    tf.summary.scalar('train_accuracy', epoch_accuracy.result().numpy(), step=step)
                    tf.summary.scalar('train_loss', epoch_loss_avg.result().numpy(), step=step)

            step += 1

        train_accuracy_results.append(epoch_accuracy.result().numpy())
        train_loss_results.append(epoch_loss_avg.result().numpy())

        for x, y in eval_seq:
            prediction = self._forward(x)
            loss_value = loss(y, prediction)
            epoch_loss_avg_val.update_state(loss_value)
            epoch_accuracy_val.update_state(y, prediction)
     
            ###############################################################
            #                                                             #
            # Добавьте сохранение метрики и функции ошибки на валидации   #
            #                                                             #
            ###############################################################
            if step % 50 == 0:
                with writer.as_default():
                    tf.summary.scalar('validation_accuracy', epoch_accuracy_val.result().numpy(), step=step)
                    tf.summary.scalar('validation_loss', epoch_loss_avg_val.result().numpy(), step=step)
            
        val_accuracy_results.append(epoch_accuracy_val.result().numpy())
        val_loss_results.append(epoch_loss_avg_val.result().numpy())

        # print(f"Epoch train loss: {epoch_train_loss[-1]:.2f},\nEpoch val loss: {epoch_val_loss[-1]:.2f}\n{'-'*20}")
        print("Epoch {}: Train loss: {:.3f} Train Accuracy: {:.3f}".format(e + 1,
                                                                           train_loss_results[-1],
                                                                           train_accuracy_results[-1]))
        print("Epoch {}: Val loss: {:.3f} Val Accuracy: {:.3f}".format(e + 1,
                                                                       val_loss_results[-1],
                                                                       val_accuracy_results[-1]))
        print('*' * 20)

    return None

In [10]:
# Monkey patch: обновляем метод
Sequential.fit_generator = fit_generator

## 2. Сравнение двух ReLU (разные активации)

Запустите два эксперимента ниже. Сравните результаты - значения метрик после каждого из них.

Запустите tensorboard, изучите распределения активаций, градиентов и т.д. для `relu` и `smart_dense_relu`. 

Подумайте, почему в одном случае сеть обучается плохо, а в другом - хорошо. Вставьте в ноутбук (или напишите список названий) тех графики из tensorboard, которые, по вашему мнению, это иллюстрируют, и напишите, почему.


Команда для запуска tensorboard в bash:

`$ tensorboard --logdir logs/`

**Ваш комментарий:**



---------




In [11]:
writer = tf.summary.create_file_writer("logs/relu")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

Epoch 1: Train loss: 11.821 Train Accuracy: 0.264
Epoch 1: Val loss: 10.622 Val Accuracy: 0.339
********************
Epoch 2: Train loss: 9.710 Train Accuracy: 0.395
Epoch 2: Val loss: 8.358 Val Accuracy: 0.478
********************
Epoch 3: Train loss: 6.681 Train Accuracy: 0.581
Epoch 3: Val loss: 5.721 Val Accuracy: 0.641
********************
Epoch 4: Train loss: 5.453 Train Accuracy: 0.658
Epoch 4: Val loss: 5.175 Val Accuracy: 0.676
********************
Epoch 5: Train loss: 5.049 Train Accuracy: 0.684
Epoch 5: Val loss: 4.776 Val Accuracy: 0.701
********************
Epoch 6: Train loss: 4.744 Train Accuracy: 0.703
Epoch 6: Val loss: 4.429 Val Accuracy: 0.722
********************
Epoch 7: Train loss: 4.261 Train Accuracy: 0.733
Epoch 7: Val loss: 3.779 Val Accuracy: 0.763
********************
Epoch 8: Train loss: 3.667 Train Accuracy: 0.769
Epoch 8: Val loss: 3.452 Val Accuracy: 0.782
********************
Epoch 9: Train loss: 3.367 Train Accuracy: 0.788
Epoch 9: Val loss: 3.206 Val 

In [ ]:
writer = tf.summary.create_file_writer("logs/relu_smart_dense")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

## 3.a Испорченный батч-норм 

Запустите два эксперимент ниже. 

Почему обучение не идет? В чем ошибка в слое `BatchNorm`? Изучите и исправьте код метода `__call__` (Шаблон находится ниже под блоком с экспериментом.).

Можно пользоваться tensorboard, если он нужен.

## ReLU + Batch Norm

In [ ]:
writer = tf.summary.create_file_writer("logs/relu_bn")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFlawed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

**Класс, который нужно исправить**

In [ ]:
class BatchNormFixed(BatchNormFlawed):
    def __call__(self, x, writer=None, step=None):
        """
        Исправьте блок кода ниже так, чтобы модель обучалась, не появлялись значения loss = NaN        """
        mu = tf.reduce_mean(x, axis=0)
        sigma = tf.math.reduce_std(x, axis=0)
        normed = (x - mu) / sigma 
        out = normed * self._gamma + self._beta
        """
        Конец блока, который нужно исправить
        """
        
        if writer is not None:
            with writer.as_default():
                tf.summary.histogram(self.name + '_beta', self._beta, step=step)
                tf.summary.histogram(self.name + '_gamma', self._gamma, step=step)
                tf.summary.histogram(self.name + '_normed', normed, step=step)
                tf.summary.histogram(self.name + '_out', out, step=step)
                tf.summary.histogram(self.name + '_sigma', sigma, step=step)
                tf.summary.histogram(self.name + '_mu', mu, step=step)
        return out

## 3.b Исправленный батч-норм 

Запустите эксперимент ниже. 

Обучается ли сеть? Идет ли процесс обучения лучше, чем в эксперименте с ReLU? 

Сравните обучение сетей c ReLU и слоем `Dense` (а не `DenseSmart`!) и ReLU с BatchNorm в tensorboard, как в задании 2.
Напишите ваши выводы.

_Обратите внимание, что слева в интерфейсе tensorboard есть меню, которое позволяет выключать визуализацию ненужных экспериментов._

**Ваш комментарий:**



---------


In [ ]:
writer = tf.summary.create_file_writer("logs/relu_bn_fixed")

model = Sequential(Dense(784, 100, tf.nn.relu, 'dense'), 
                   BatchNormFixed('batch_norm'), 
                   Dense(100, 100, tf.nn.relu, 'dense1'), 
                   Dense(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq, test_seq, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer)

## 4. "Сырые" данные. 

Что будет, если заставить сеть обучаться на сырых данных? 

Напишите такую функцию `preprocess`, которая не делает min-max scaling изображений и оставляет их в изначальном диапазоне. Не убирайте reshape! Конечно, она должна менять форму матрицы входных данных от `(n x 28 x 28)` к `(n x 784)`. 

Затем передайте функцию в MNISTSequence, создайте новую train- и test- последовательности запустите эксперимент, используя их как входные данные. 

Сравните результаты экспериментов c `DenseSmart` + ReLU и обработанными изображениями и `DenseSmart` + ReLU c необработанными изображениями. 

Обучается ли нейросеть? Если нет, то почему? Сделайте выводы, как в задании 2.

**Ваш комментарий:**



---------


**Шаблон Preprocess**

In [ ]:
def preprocess(X, y):
    """
    Ваш код
    """
    return X, y

**Создание генераторов**

In [ ]:
train_seq_raw = MNISTSequence(...)
test_seq_raw = MNISTSequence(...)

**Эксперимент**

In [ ]:
writer = tf.summary.create_file_writer("logs/raw")

model = Sequential(DenseSmart(784, 100, tf.nn.relu, 'dense'), 
                   DenseSmart(100, 100, tf.nn.relu, 'dense1'), 
                   DenseSmart(100, 10, tf.nn.softmax, 'dense2'))

hist = model.fit_generator(train_seq_raw, test_seq_raw, 10,
                           keras.losses.sparse_categorical_crossentropy, 
                           keras.optimizers.Adam(),
                           writer
                          )